<a href="https://colab.research.google.com/github/ashikshafi08/AICrowd_Competition_Files/blob/main/blitz_faces/AICrowd_sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is work in progress still....

Just setting up a baseline model (with tensorflow)

## Setting up AI-Crowd CLI 


In [1]:
!pip install aicrowd-cli
%load_ext aicrowd.magic

%aicrowd login

# Downloading the dataset and making directories 

!rm -rf data
!mkdir data
%aicrowd ds dl -c sentiment-classification -o data

     |████████████████████████████████| 48 kB 2.4 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
     |████████████████████████████████| 54 kB 2.7 MB/s 
     |████████████████████████████████| 170 kB 35.1 MB/s 
     |████████████████████████████████| 1.1 MB 60.2 MB/s 
     |████████████████████████████████| 214 kB 46.4 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
     |████████████████████████████████| 51 kB 5.1 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: pyzmq
    Found existing installation: pyzmq 22.3.0
    Uninstalling pyzmq-22.3.0:
      Successfully uninstalled pyzmq-22.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you hav

Please login here: https://api.aicrowd.com/auth/qYVtKAPJV3a_N_kpa8gkRuSSy1ZbEd5gL8PlBqwV2oA
API Key valid
Gitlab access token valid
Saved details successfully!


sample_submission.csv:   0%|          | 0.00/30.9M [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/30.9M [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/51.5M [00:00<?, ?B/s]

val.csv:   0%|          | 0.00/20.6M [00:00<?, ?B/s]

In [73]:
!pip uninstall rich

Found existing installation: rich 10.16.2
Uninstalling rich-10.16.2:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/rich-10.16.2.dist-info/*
    /usr/local/lib/python3.7/dist-packages/rich/*
Proceed (y/n)? y
  Successfully uninstalled rich-10.16.2


### Working with the data 

In [74]:
# Importing the data 
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 

# Setting up the path for the data 
train_path = '/content/data/train.csv'
test_path = '/content/data/test.csv'
sam_path = '/content/data/sample_submission.csv'
val_path = '/content/data/val.csv'

# Loading in the data
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)
val_df = pd.read_csv(val_path)
sample_sub_df = pd.read_csv(sam_path)

train_df.head()

,embeddings,label
0,"[0.3206779360771179, 0.988215982913971, 1.0441...",positive
1,"[0.05074610561132431, 1.0742985010147095, 0.60...",negative
2,"[0.41962647438049316, 0.4505457878112793, 1.39...",negative
3,"[0.4361684024333954, 0.19191382825374603, 0.83...",positive
4,"[0.6382085084915161, 0.8352395296096802, 0.393...",neutral


[1, 0, 0, 1]
['positive' 'negative' 'negative' 'positive']


In [75]:
# Looking at one embedding 
print(train_df['embeddings'][0])

[0.3206779360771179, 0.988215982913971, 1.0441516637802124, 0.6625359654426575, 0.3325623869895935, 0.008159726858139038, 0.743507444858551, 1.7079613208770752, 0.7912898063659668, 0.26170486211776733, 0.22327399253845215, 0.14752592146396637, 1.353232502937317, 0.2673041820526123, 0.17822319269180298, 0.7025171518325806, 0.5975980162620544, 0.9886007308959961, 0.1991458535194397, 0.26832789182662964, 0.1274198442697525, 1.3344076871871948, 0.22078244388103485, 0.20043855905532837, 0.7809276580810547, 0.26213347911834717, 0.17527638375759125, 0.4533098042011261, 0.12350031733512878, 0.4940362572669983, 1.0229181051254272, 0.35457369685173035, 0.00927578192204237, 1.7569537162780762, 0.6719991564750671, 0.9060824513435364, 0.9972180724143982, 0.32104337215423584, 1.1657556295394897, 0.24956151843070984, 0.36764925718307495, 0.294410765171051, 0.6678937077522278, 0.5075835585594177, 0.7074298858642578, 1.7918670177459717, 0.7123008966445923, 0.7682262063026428, 2.3446991443634033, 1.2260

In [76]:
# Return a list of embedding (numpy) array
from ast import literal_eval

def convert_to_array(df):

    eval_lst = [literal_eval(emb) for emb in df['embeddings'].values]
    lst_nump_arr = [np.array(arr, dtype = np.float64) for arr in eval_lst]
    return lst_nump_arr

In [206]:
# Converting the whole dataframe 
X = convert_to_array(df = train_df)
y = [int(elem == 'positive') for elem in train_df['label'].values] 

# Positive - 1 , neg - 0

X_val = convert_to_array(val_df)
y_val = [int(elem == 'positive') for elem in val_df['label'].values]

#y_val = tf.expand_dims(tf.convert_to_tensor(y_val , dtype= tf.string) , axis = 1)



In [207]:
import tensorflow as tf 
from tensorflow.keras import layers 


# Creating a tennsorflow dataset 
train_dataset = tf.data.Dataset.from_tensor_slices((X , y))
val_dataset = tf.data.Dataset.from_tensor_slices((X_val , y_val))

# Applying batch and prefetch 
train_dataset = train_dataset.batch(128).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.prefetch(tf.data.AUTOTUNE).batch(1)

In [208]:
train_dataset , val_dataset

(<PrefetchDataset shapes: ((None, 512), (None,)), types: (tf.float64, tf.int32)>,
 <BatchDataset shapes: ((None, 512), (None,)), types: (tf.float64, tf.int32)>)

In [219]:
# Building a small model 

input_layer = layers.Input(shape = (512,))
#x = layers.Lambda(lambda x: tf.expand_dims(x, axis = 1))(input_layer)
x = layers.Dense(64 , activation = 'relu')(input_layer)
x = layers.Dense(10 , activation = 'relu')(x)

output_layer = layers.Dense(1 , activation = 'sigmoid')(x)

# Packing into a model 
model = tf.keras.Model(input_layer , output_layer)
model.summary()

# model = tf.keras.Sequential([
#     layers.Dense(32, activation='relu'), 
#     layers.Dense(10 , activation = 'relu'), 
#     layers.Dense(1 , activation = 'sigmoid')
# ])

Model: "model_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_17 (InputLayer)       [(None, 512)]             0         
                                                                 
 dense_103 (Dense)           (None, 64)                32832     
                                                                 
 dense_104 (Dense)           (None, 10)                650       
                                                                 
 dense_105 (Dense)           (None, 1)                 11        
                                                                 
Total params: 33,493
Trainable params: 33,493
Non-trainable params: 0
_________________________________________________________________


In [220]:
model.compile(loss = tf.keras.losses.BinaryCrossentropy() , 
              optimizer = tf.keras.optimizers.Adam(learning_rate = .0001) , 
              metrics = ['accuracy'])

# Fitting the model 
model.fit(train_dataset, validation_data = val_dataset , epochs = 10) 

Epoch 1/10
40/40 [==============================] - 3s 72ms/step - loss: 0.5314 - accuracy: 0.7242 - val_loss: 0.4997 - val_accuracy: 0.7425
Epoch 2/10
40/40 [==============================] - 3s 69ms/step - loss: 0.4448 - accuracy: 0.7852 - val_loss: 0.4479 - val_accuracy: 0.7915
Epoch 3/10
40/40 [==============================] - 3s 69ms/step - loss: 0.4025 - accuracy: 0.8152 - val_loss: 0.4195 - val_accuracy: 0.8140
Epoch 4/10
40/40 [==============================] - 3s 69ms/step - loss: 0.3757 - accuracy: 0.8296 - val_loss: 0.3999 - val_accuracy: 0.8210
Epoch 5/10
40/40 [==============================] - 2s 52ms/step - loss: 0.3562 - accuracy: 0.8406 - val_loss: 0.3850 - val_accuracy: 0.8290
Epoch 6/10
40/40 [==============================] - 3s 69ms/step - loss: 0.3408 - accuracy: 0.8504 - val_loss: 0.3727 - val_accuracy: 0.8355
Epoch 7/10
40/40 [==============================] - 2s 54ms/step - loss: 0.3279 - accuracy: 0.8568 - val_loss: 0.3618 - val_accuracy: 0.8415
Epoch 8/10
40